In [1]:
import Io_tf_binary_general as io

Welcome to JupyROOT 6.15/01


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import read_tree as rt

In [18]:
def wczytaj_do_pythonowych(sciezka_rootowego):
    """funkcja zwraca pythonowe w wygodnej do uzycia formie"""
    legs, jets, global_params, properties = rt.read_tree(file_name=sciezka_rootowego)
    #te rzeczy sa zle, bo maja niewygodny format oraz global_params["nJets30"] ma
    #zdublowane dane, to znaczy [1,1,3,3,5,5 ...]
    #mozna to sprawdzic ta metoda
    zdublowany_klucz="nJets30"
    def czy_zdublowane(tab):
        for i in range(len(tab)//2):
            if tab[2*i] != tab[2*i+1]:
                print ("kontrprzylad dla tezy, ze to jest zdublowana lista")
                assert False
    czy_zdublowane(global_params[zdublowany_klucz])
    def przerob_legs_lub_jets(tab):
        tab=np.array(tab)
        return np.transpose(tab,(2,0,1))
    def przerob_global_params_lub_properties(slownik):
        wyrzut=[]
        jeden_z_kluczy=list(slownik.keys())[0]
        #uwaga hakerstwo
        liczba_przypadkow=0
        if jeden_z_kluczy==zdublowany_klucz:
            liczba_przypadkow=len(slownik[jeden_z_kluczy]) //2
        else:
            liczba_przypadkow=len(slownik[jeden_z_kluczy])
        for i in range(liczba_przypadkow):
            doczepiany={}
            for k in slownik.keys():
                if k != zdublowany_klucz:
                    doczepiany[k]=slownik[k][i]
                else:
                    doczepiany[k]=slownik[k][2*i]
            wyrzut.append(doczepiany)
        return wyrzut
    legs=przerob_legs_lub_jets(legs)
    jets=przerob_legs_lub_jets(jets)
    global_params=przerob_global_params_lub_properties(global_params)
    properties=przerob_global_params_lub_properties(properties)
    return legs, jets, global_params, properties


In [19]:
legs_syg, jets_syg, global_params_syg, properties_syg=\
wczytaj_do_pythonowych("data_rootowe_obrobione/MT_GluGlu.root")

[ML]	Opening root file for python conversion.
[ML]	Reading data from TTree.
[ML]	Conversion to python successful!


In [20]:
legs_tlo, jets_tlo, global_params_tlo, properties_tlo=\
wczytaj_do_pythonowych("data_rootowe_obrobione/MT_WJets.root")

[ML]	Opening root file for python conversion.
[ML]	Reading data from TTree.
[ML]	Conversion to python successful!


In [21]:
liczba_przypadkow_tlo=len(legs_tlo)
liczba_przypadkow_syg=len(legs_syg)

In [22]:
print(liczba_przypadkow_tlo)
print(liczba_przypadkow_syg)

3087
59083


In [23]:
#mozna tu zauwazyc pewna dysproporcje. 
#na szybko mozna ta sytuacje rozwiazac uzywajac wielokrotnie przypadkow tla, tak by
#bylo mniej wiecej po rowno tych rzeczy

In [24]:
zapisywacz_syg=io.Io_tf_binary_general("sygnal",'w')

In [25]:
zapisywacz_tlo=io.Io_tf_binary_general("tlo",'w')

In [26]:
for i in range(liczba_przypadkow_syg):
    zapisywacz_syg.\
    write_from_tree(legs_syg[i],jets_syg[i],global_params_syg[i],
        properties_syg[i],1)

In [27]:
for i in range(liczba_przypadkow_tlo):
    zapisywacz_tlo.\
    write_from_tree(legs_tlo[i],jets_tlo[i],global_params_tlo[i],
        properties_tlo[i],0)